In [ ]:
import google

In [ ]:
!pip install googlesearch-python

In [ ]:
!pip install elasticsearch==5.5.3

In [ ]:
from googlesearch import search
query = "Migration"
for result in search(query, lang='en'):
      print (result)

In [ ]:
import urllib.request as url
url_link = "https://www.iom.int/who-migrant-0"
url_content = url.urlopen(url_link).read()

In [ ]:
# convert it to legible text
from bs4 import BeautifulSoup
text_content = BeautifulSoup(url_content, "lxml").text